In [ ]:
# Instala librerías necesarias
%pip install pandas pyreadstat matplotlib scipy seaborn

# Análisis Exploratorio de Datos - Defunciones Guatemala (2009-2020)
## Inciso a: Descripción del Conjunto de Datos

Este notebook analiza los datos de defunciones en Guatemala de 2009 a 2020, describiendo:
- Número de variables y observaciones
- Tipo de cada variable
- Estructura general del dataset unificado

## 1. Importar Librerías Necesarias

In [ ]:
import pandas as pd
import numpy as np
import pyreadstat
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas exitosamente")

## 2. Cargar y Explorar Archivos .sav Individuales

Primero, identificamos todos los archivos .sav disponibles y exploramos su estructura.

In [ ]:
# Define la ruta a los datos (subir un nivel desde la carpeta actual)
data_path = Path(r'../data/defunciones/sav')

# Lista todos los archivos .sav
archivos_sav = sorted(list(data_path.glob('*.sav')))

print(f"Total de archivos .sav encontrados: {len(archivos_sav)}\n")
print("Archivos disponibles:")
for i, archivo in enumerate(archivos_sav, 1):
    print(f"{i}. {archivo.name}")

In [ ]:
# Explora un archivo de ejemplo para entender la estructura
ejemplo_archivo = archivos_sav[0]
print(f"Cargando archivo de ejemplo: {ejemplo_archivo.name}\n")

# Carga el archivo .sav con metadatos
df_ejemplo, meta = pyreadstat.read_sav(str(ejemplo_archivo))

print(f"Dimensiones: {df_ejemplo.shape[0]} observaciones x {df_ejemplo.shape[1]} variables")
print(f"\nPrimeras 5 filas:")
display(df_ejemplo.head())

print(f"\nInformación del DataFrame:")
display(df_ejemplo.info())

## 3. Analizar Consistencia de Variables entre Años

Verificamos si las variables son consistentes entre todos los años o si hay cambios.

In [ ]:
# Carga todos los archivos y revisar estructura
estructura_por_anio = {}

for archivo in archivos_sav:
    year = archivo.stem.split('-')[0]  # Extrae el año del nombre del archivo
    df_temp, meta_temp = pyreadstat.read_sav(str(archivo))
    
    estructura_por_anio[year] = {
        'num_obs': df_temp.shape[0],
        'num_vars': df_temp.shape[1],
        'columnas': list(df_temp.columns),
        'tipos': df_temp.dtypes.to_dict()
    }

# Crea DataFrame de resumen
resumen_estructura = pd.DataFrame({
    'Año': list(estructura_por_anio.keys()),
    'Observaciones': [estructura_por_anio[y]['num_obs'] for y in estructura_por_anio.keys()],
    'Variables': [estructura_por_anio[y]['num_vars'] for y in estructura_por_anio.keys()]
})

print("Resumen de estructura por año:")
display(resumen_estructura)

# Verifica consistencia de columnas
todas_columnas = [set(estructura_por_anio[y]['columnas']) for y in estructura_por_anio.keys()]
columnas_comunes = set.intersection(*todas_columnas)
columnas_todas = set.union(*todas_columnas)

print(f"\nTotal de columnas únicas en todos los archivos: {len(columnas_todas)}")
print(f"Columnas comunes a todos los años: {len(columnas_comunes)}")
print(f"Columnas que varían entre años: {len(columnas_todas - columnas_comunes)}")

if columnas_todas != columnas_comunes:
    print("\n Las columnas NO son idénticas en todos los años. Habrá que manejar diferencias al unificar.")

## 4. Unificar Datos de Todos los Años

Como necesitamos trabajar con más de 10 años de datos (según las instrucciones), procedemos a unificar todos los archivos en un solo DataFrame.

In [ ]:
# Unifica todos los archivos
dataframes = []
metadatos = {}

for archivo in archivos_sav:
    year = archivo.stem.split('-')[0]
    print(f"Cargando: {archivo.name}...")
    
    df_temp, meta_temp = pyreadstat.read_sav(str(archivo))
    
    # Agrega columna de año
    df_temp['ANIO'] = int(year)
    
    dataframes.append(df_temp)
    metadatos[year] = meta_temp

# Concatena todos los DataFrames
df_unificado = pd.concat(dataframes, axis=0, ignore_index=True, sort=False)

print(f"\n Datos unificados exitosamente!")
print(f"Total de observaciones: {df_unificado.shape[0]:,}")
print(f"Total de variables: {df_unificado.shape[1]:,}")
print(f"Años incluidos: {sorted(df_unificado['ANIO'].unique())}")

## 5. Conteo de Variables y Observaciones

Resumen del conjunto de datos unificado.

In [ ]:
# Resumen por año
observaciones_por_anio = df_unificado.groupby('ANIO').size().reset_index(name='Observaciones')
print("Distribución de observaciones por año:")
display(observaciones_por_anio)

# Estadísticas generales
print(f"\n{'='*60}")
print(f"RESUMEN DEL CONJUNTO DE DATOS UNIFICADO")
print(f"{'='*60}")
print(f"Total de observaciones (registros): {df_unificado.shape[0]:,}")
print(f"Total de variables (columnas): {df_unificado.shape[1]:,}")
print(f"Período cubierto: {df_unificado['ANIO'].min()} - {df_unificado['ANIO'].max()}")
print(f"Años de datos: {df_unificado['ANIO'].nunique()} años")
print(f"{'='*60}")

## 6. Clasificación de Tipos de Variables

Analizamos cada variable para clasificarla como numérica (continua/discreta) o categórica.

In [ ]:
# Clasifica variables según teoría estadística
# Categóricas: Nominales (sin orden) y Ordinales (con orden)
# Numéricas: Continuas (cualquier valor) y Discretas (valores contables)

clasificacion_vars = []

# Define variables que son categóricas aunque estén codificadas como números
variables_categoricas_codificadas = {
    'Sexo': 'Categórica Nominal',  # 1, 2 pero sin orden intrínseco
    'Depreg': 'Categórica Nominal',  # Códigos de departamentos son categorías
    'Depocu': 'Categórica Nominal',
    'Areag': 'Categórica Nominal',  # Área geográfica (urbano/rural)
    'Puedif': 'Categórica Nominal',  # Pueblo de pertenencia
    'Pnadif': 'Categórica Nominal',  # País de nacimiento
    'Dnadif': 'Categórica Nominal',  # Departamento de nacimiento
    'Nacdif': 'Categórica Nominal',  # Nacionalidad
    'Predif': 'Categórica Nominal',  # País de residencia
    'Dredif': 'Categórica Nominal',  # Departamento de residencia
    'Ocur': 'Categórica Nominal',  # Sitio de ocurrencia
    'Getdif': 'Categórica Nominal',  # Grupo étnico
}

# Variables ordinales (tienen orden lógico)
variables_ordinales = {
    'Escodif': 'Categórica Ordinal',  # Escolaridad del difunto(a) - tiene orden: ninguno, primaria, secundaria, etc.
    'Ecidif': 'Categórica Ordinal',  # Estado civil del difunto(a) - puede tener orden lógico
    'Perdif': 'Categórica Ordinal',  # Periodo de edad del difunto(a) - orden: horas, días, meses, años
    'Asist': 'Categórica Ordinal',  # Asistencia recibida - puede tener orden según nivel de atención
    'Cerdef': 'Categórica Ordinal',  # Quien certifica - puede tener orden según nivel de autoridad
}

for columna in df_unificado.columns:
    dtype = df_unificado[columna].dtype
    num_unicos = df_unificado[columna].nunique()
    valores_ejemplo = df_unificado[columna].dropna().head(3).tolist()
    
    # Clasifica tipo de variable
    if columna in variables_categoricas_codificadas:
        tipo_var = variables_categoricas_codificadas[columna]
    elif columna in variables_ordinales:
        tipo_var = variables_ordinales[columna]
    elif dtype in ['int64', 'float64']:
        # Numéricas
        if num_unicos <= 20:
            tipo_var = 'Numérica Discreta'
        else:
            tipo_var = 'Numérica Continua'
    else:
        # Categóricas (tipo object/string)
        col_lower = columna.lower()
        if any(keyword in col_lower for keyword in ['escol', 'nivel', 'grado', 'civil']):
            tipo_var = 'Categórica Ordinal'
        else:
            tipo_var = 'Categórica Nominal'
    
    clasificacion_vars.append({
        'Variable': columna,
        'Tipo_Dato': str(dtype),
        'Tipo_Variable': tipo_var,
        'Valores_Únicos': num_unicos,
        'Ejemplo_Valores': str(valores_ejemplo[:3])
    })

df_clasificacion = pd.DataFrame(clasificacion_vars)

# Muestra resumen
print("="*80)
print("CLASIFICACIÓN DE VARIABLES (SEGÚN TEORÍA ESTADÍSTICA)")
print("="*80)
print("\nResumen por tipo de variable:")
print(df_clasificacion['Tipo_Variable'].value_counts().sort_index())
print(f"\nTotal de variables: {len(df_clasificacion)}")

# Desglose por categoría
print("\n" + "="*80)
print("VARIABLES CATEGÓRICAS NOMINALES:")
print("-"*80)
nominales = df_clasificacion[df_clasificacion['Tipo_Variable'] == 'Categórica Nominal']
print(f"Total: {len(nominales)} variables")
print("(Sin orden intrínseco: ej. departamento, municipio, sexo)")
if len(nominales) > 0:
    for var in nominales['Variable'].tolist():
        print(f"  • {var}")

print("\n" + "="*80)
print("VARIABLES CATEGÓRICAS ORDINALES:")
print("-"*80)
ordinales = df_clasificacion[df_clasificacion['Tipo_Variable'] == 'Categórica Ordinal']
print(f"Total: {len(ordinales)} variables")
print("(Con orden lógico: ej. escolaridad, estado civil)")
if len(ordinales) > 0:
    for var in ordinales['Variable'].tolist():
        print(f"  • {var}")

print("\n" + "="*80)
print("VARIABLES NUMÉRICAS CONTINUAS:")
print("-"*80)
continuas = df_clasificacion[df_clasificacion['Tipo_Variable'] == 'Numérica Continua']
print(f"Total: {len(continuas)} variables")
print("(Pueden tomar cualquier valor en un rango: ej. edad exacta)")
if len(continuas) > 0 and len(continuas) <= 15:
    for var in continuas['Variable'].tolist():
        print(f"  • {var}")

print("\n" + "="*80)
print("VARIABLES NUMÉRICAS DISCRETAS:")
print("-"*80)
discretas = df_clasificacion[df_clasificacion['Tipo_Variable'] == 'Numérica Discreta']
print(f"Total: {len(discretas)} variables")
print("(Valores enteros contables: ej. mes, año, día)")
if len(discretas) > 0 and len(discretas) <= 15:
    for var in discretas['Variable'].tolist():
        print(f"  • {var}")

# Muestra tabla completa
print("\n" + "="*80)
print("DETALLE COMPLETO DE TODAS LAS VARIABLES:")
print("="*80)
display(df_clasificacion)

## 7. Tabla de Resumen Completo con Metadatos

Generamos una tabla comprehensiva con información detallada de cada variable, incluyendo etiquetas de los archivos SPSS.

In [ ]:
# Obtiene etiquetas de variables desde los metadatos SPSS
meta_ejemplo = metadatos[list(metadatos.keys())[0]]  # Usar metadatos del primer año

resumen_completo = []

for columna in df_unificado.columns:
    # Obtiene etiqueta desde metadatos si existe
    etiqueta = meta_ejemplo.column_names_to_labels.get(columna, 'Sin descripción')
    
    # Información básica
    num_missing = df_unificado[columna].isna().sum()
    pct_missing = (num_missing / len(df_unificado)) * 100
    num_unicos = df_unificado[columna].nunique()
    
    # Tipo de variable
    tipo_fila = df_clasificacion[df_clasificacion['Variable'] == columna]['Tipo_Variable'].values
    tipo_var = tipo_fila[0] if len(tipo_fila) > 0 else 'N/A'
    
    resumen_completo.append({
        'Variable': columna,
        'Descripción': etiqueta,
        'Tipo': tipo_var,
        'Tipo_Dato': str(df_unificado[columna].dtype),
        'Valores_Únicos': num_unicos,
        'Valores_Faltantes': num_missing,
        '% Faltantes': f"{pct_missing:.2f}%"
    })

df_resumen = pd.DataFrame(resumen_completo)

print("TABLA RESUMEN COMPLETA DE VARIABLES")
print("="*80)
display(df_resumen)

# Guarda a CSV para referencia
df_resumen.to_csv('resumen_variables_defunciones.csv', index=False, encoding='utf-8-sig')
print("\n Tabla guardada en: resumen_variables_defunciones.csv")

## 8. Visualización de la Distribución de Tipos de Variables

In [ ]:
import matplotlib.pyplot as plt

# Cuenta tipos de variables (usando la clasificación correcta de df_clasificacion)
conteo_tipos = df_clasificacion['Tipo_Variable'].value_counts().sort_index()

# Crea visualización con un solo gráfico de barras
fig, ax = plt.subplots(figsize=(12, 6))

# Colores para cada tipo de variable
colors_dict = {
    'Categórica Nominal': '#3498db',
    'Categórica Ordinal': '#9b59b6',
    'Numérica Continua': '#2ecc71',
    'Numérica Discreta': '#f39c12'
}
colors = [colors_dict.get(tipo, '#95a5a6') for tipo in conteo_tipos.index]

# Gráfico de barras horizontales
ax.barh(conteo_tipos.index, conteo_tipos.values, color=colors)
ax.set_xlabel('Cantidad de Variables', fontsize=12, fontweight='bold')
ax.set_ylabel('Tipo de Variable', fontsize=12, fontweight='bold')
ax.set_title('Distribución de Tipos de Variables\n(Clasificación Estadística)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Añade valores en las barras
for i, v in enumerate(conteo_tipos.values):
    ax.text(v + 0.3, i, str(v), va='center', fontweight='bold', fontsize=11)

plt.tight_layout()

# Guarda gráfico - INCISO A

plt.show()

# Resumen estadístico detallado
print("\n" + "="*80)
print("RESUMEN ESTADÍSTICO DE TIPOS DE VARIABLES:")
print("="*80)
for tipo, cantidad in conteo_tipos.items():
    porcentaje = (cantidad / len(df_clasificacion)) * 100
    print(f"  • {tipo}: {cantidad} variables ({porcentaje:.1f}%)")

print("\n" + "="*80)
print("INTERPRETACIÓN:")
print("="*80)
print("  • Variables Categóricas Nominales: Sin orden intrínseco")
print("    Ejemplos: Departamento, Municipio, Sexo, Causa de muerte")
print("\n  • Variables Categóricas Ordinales: Con orden lógico")
print("    Ejemplos: Nivel de escolaridad, Estado civil")
print("\n  • Variables Numéricas Continuas: Pueden tomar cualquier valor en un rango")
print("    Ejemplos: Edad exacta, mediciones continuas")
print("\n  • Variables Numéricas Discretas: Valores enteros contables")
print("    Ejemplos: Mes, Año, Día, conteos")
print("="*80)

## 9. Análisis de Valores Faltantes

Identificamos variables con alta proporción de datos faltantes.

In [ ]:
# Variables con valores faltantes
df_missing = df_resumen[df_resumen['Valores_Faltantes'] > 0].sort_values('Valores_Faltantes', ascending=False)

print(f"Variables con valores faltantes: {len(df_missing)} de {len(df_resumen)}")
print(f"\nTop 10 variables con más valores faltantes:")
display(df_missing.head(10))

# Visualizar
if len(df_missing) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    top_missing = df_missing.head(15)
    ax.barh(top_missing['Variable'], top_missing['Valores_Faltantes'], color='coral')
    ax.set_xlabel('Cantidad de Valores Faltantes', fontsize=12)
    ax.set_ylabel('Variable', fontsize=12)
    ax.set_title('Top 15 Variables con Más Valores Faltantes', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Guarda gráfico - INCISO A
    
    plt.show()

## Conclusiones del Inciso a

### Resumen de hallazgos:

In [ ]:
print("="*80)
print("CONCLUSIONES - INCISO A: DESCRIPCIÓN DEL CONJUNTO DE DATOS")
print("="*80)
print()
print("1. DIMENSIONES DEL DATASET:")
print(f"   - Total de observaciones: {df_unificado.shape[0]:,} registros de defunciones")
print(f"   - Total de variables: {df_unificado.shape[1]} variables")
print(f"   - Período: {df_unificado['ANIO'].min()}-{df_unificado['ANIO'].max()} ({df_unificado['ANIO'].nunique()} años)")
print()

print("2. CLASIFICACIÓN DE VARIABLES:")
for tipo, cantidad in df_resumen['Tipo'].value_counts().items():
    porcentaje = (cantidad / len(df_resumen)) * 100
    print(f"   - {tipo}: {cantidad} variables ({porcentaje:.1f}%)")
print()

print("3. CALIDAD DE DATOS:")
total_con_missing = len(df_resumen[df_resumen['Valores_Faltantes'] > 0])
pct_con_missing = (total_con_missing / len(df_resumen)) * 100
print(f"   - Variables con datos faltantes: {total_con_missing} ({pct_con_missing:.1f}%)")
print(f"   - Variables completas: {len(df_resumen) - total_con_missing}")
print()

# Inciso b: Análisis Estadístico de Variables

## Resumen de Variables Numéricas y Pruebas de Normalidad

En esta sección analizaremos:
- Estadísticas descriptivas de las variables numéricas
- Pruebas de normalidad (Shapiro-Wilk, Kolmogorov-Smirnov)
- Identificación del tipo de distribución
- Tablas de frecuencia para variables categóricas

## 1. Separar Variables Numéricas y Categóricas

Primero identificamos y separamos las variables numéricas de las categóricas.

In [ ]:
# Separa variables numéricas y categóricas
variables_numericas = df_unificado.select_dtypes(include=['int64', 'float64']).columns.tolist()
variables_categoricas = df_unificado.select_dtypes(include=['object']).columns.tolist()

print(f"Variables Numéricas ({len(variables_numericas)}):")
print(variables_numericas)
print(f"\nVariables Categóricas ({len(variables_categoricas)}):")
print(variables_categoricas)

## 2. Resumen Estadístico de Variables Numéricas

Calculamos las estadísticas descriptivas principales para todas las variables numéricas.

In [ ]:
# Resumen estadístico completo de variables numéricas
print("="*100)
print("RESUMEN ESTADÍSTICO DE VARIABLES NUMÉRICAS")
print("="*100)

resumen_estadistico = df_unificado[variables_numericas].describe().T

# Añade estadísticas adicionales
resumen_estadistico['mediana'] = df_unificado[variables_numericas].median()
resumen_estadistico['moda'] = df_unificado[variables_numericas].mode().iloc[0]
resumen_estadistico['asimetria'] = df_unificado[variables_numericas].skew()
resumen_estadistico['curtosis'] = df_unificado[variables_numericas].kurtosis()
resumen_estadistico['missing'] = df_unificado[variables_numericas].isna().sum()
resumen_estadistico['missing_%'] = (df_unificado[variables_numericas].isna().sum() / len(df_unificado) * 100).round(2)

# Reordena columnas
resumen_estadistico = resumen_estadistico[['count', 'mean', 'mediana', 'moda', 'std', 'min', '25%', '50%', '75%', 'max', 'asimetria', 'curtosis', 'missing', 'missing_%']]

display(resumen_estadistico)

## 3. Pruebas de Normalidad

Para evaluar si las variables numéricas siguen una distribución normal, utilizaremos:
- **Test de Shapiro-Wilk** (para muestras pequeñas, usaremos una muestra aleatoria)
- **Test de Kolmogorov-Smirnov** (para muestras grandes)
- **Análisis de asimetría y curtosis**
- **Gráficos Q-Q Plot y histogramas**

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Configura estilo de gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 6)

# Realiza pruebas de normalidad
resultados_normalidad = []

print("="*100)
print("PRUEBAS DE NORMALIDAD PARA VARIABLES NUMÉRICAS")
print("="*100)
print("\nNota: Usando muestra de 5000 registros para Shapiro-Wilk (limitación del test)\n")

for variable in variables_numericas:
    # Remueve valores nulos
    datos_limpios = df_unificado[variable].dropna()
    
    if len(datos_limpios) > 0:
        # Toma muestra para Shapiro-Wilk (máximo 5000 por limitaciones del test)
        muestra = datos_limpios.sample(min(5000, len(datos_limpios)), random_state=42)
        
        # Test de Shapiro-Wilk
        if len(muestra) >= 3:
            shapiro_stat, shapiro_p = stats.shapiro(muestra)
        else:
            shapiro_stat, shapiro_p = None, None
        
        # Test de Kolmogorov-Smirnov
        ks_stat, ks_p = stats.kstest(datos_limpios, 'norm', args=(datos_limpios.mean(), datos_limpios.std()))
        
        # Asimetría y Curtosis
        asimetria = datos_limpios.skew()
        curtosis = datos_limpios.kurtosis()
        
        # Clasificación de normalidad
        es_normal = (shapiro_p > 0.05 if shapiro_p else False) and (ks_p > 0.05)
        
        resultados_normalidad.append({
            'Variable': variable,
            'N': len(datos_limpios),
            'Shapiro_W': shapiro_stat,
            'Shapiro_p': shapiro_p,
            'KS_stat': ks_stat,
            'KS_p': ks_p,
            'Asimetría': asimetria,
            'Curtosis': curtosis,
            'Es_Normal': 'Sí' if es_normal else 'No'
        })

# Crea DataFrame de resultados
df_normalidad = pd.DataFrame(resultados_normalidad)

# Formatea para mejor visualización
df_normalidad_display = df_normalidad.copy()
df_normalidad_display['Shapiro_p'] = df_normalidad_display['Shapiro_p'].apply(lambda x: f"{x:.4e}" if x is not None else 'N/A')
df_normalidad_display['KS_p'] = df_normalidad_display['KS_p'].apply(lambda x: f"{x:.4e}")
df_normalidad_display['Asimetría'] = df_normalidad_display['Asimetría'].apply(lambda x: f"{x:.2f}")
df_normalidad_display['Curtosis'] = df_normalidad_display['Curtosis'].apply(lambda x: f"{x:.2f}")

display(df_normalidad_display)

# Resumen de normalidad
print(f"\n{'='*100}")
print("RESUMEN DE NORMALIDAD:")
print(f"{'='*100}")
normal_count = (df_normalidad['Es_Normal'] == 'Sí').sum()
no_normal_count = (df_normalidad['Es_Normal'] == 'No').sum()
print(f"Variables que siguen distribución normal: {normal_count} ({normal_count/len(df_normalidad)*100:.1f}%)")
print(f"Variables que NO siguen distribución normal: {no_normal_count} ({no_normal_count/len(df_normalidad)*100:.1f}%)")

### Interpretación de Asimetría y Curtosis

- **Asimetría (Skewness)**:
  - = 0: Distribución simétrica
  - \> 0: Cola derecha (sesgo positivo)
  - < 0: Cola izquierda (sesgo negativo)
  
- **Curtosis**:
  - = 0: Similar a distribución normal
  - \> 0: Leptocúrtica (colas más pesadas, pico más alto)
  - < 0: Platicúrtica (colas más ligeras, pico más bajo)

## 4. Visualización de Distribuciones - Variables Numéricas Clave

Analizaremos gráficamente las variables numéricas más importantes.

In [ ]:
# Selecciona variables clave para visualización detallada
variables_clave = ['Edadif', 'Sexo', 'Mesreg', 'Mesocu', 'Depreg', 'Depocu', 'ANIO']

# Define rangos válidos para cada variable (excluir códigos de missing como 999, 888, etc.)
rangos_validos = {
    'Edadif': (0, 120),      # Edad razonable para humanos
    'Sexo': (1, 2),          # 1=Masculino, 2=Femenino
    'Mesreg': (1, 12),       # Meses del año
    'Mesocu': (1, 12),       # Meses del año
    'Depreg': (1, 22),       # Departamentos de Guatemala
    'Depocu': (1, 22),       # Departamentos de Guatemala
    'ANIO': (2009, 2020)     # Rango de años del estudio
}

# Filtra solo las que existen en el dataset
variables_clave = [v for v in variables_clave if v in df_unificado.columns]

print(f"Analizando {len(variables_clave)} variables clave: {variables_clave}\n")
print("="*100)

# Crea subplots para cada variable clave
for idx, variable in enumerate(variables_clave, start=1):
    datos_originales = df_unificado[variable].dropna()
    
    if len(datos_originales) == 0:
        continue
    
    # Aplica filtro de rango válido
    if variable in rangos_validos:
        min_val, max_val = rangos_validos[variable]
        datos = datos_originales[(datos_originales >= min_val) & (datos_originales <= max_val)]
        excluidos = len(datos_originales) - len(datos)
        pct_excluidos = (excluidos / len(datos_originales)) * 100
    else:
        datos = datos_originales
        excluidos = 0
        pct_excluidos = 0
    
    if len(datos) == 0:
        print(f"⚠️  {variable}: Todos los datos fueron filtrados")
        continue
    
    # Determina número apropiado de bins según el rango
    n_unicos = datos.nunique()
    if n_unicos <= 25:
        n_bins = n_unicos  # Para variables discretas con pocos valores
    else:
        n_bins = min(50, int(np.sqrt(len(datos))))  # Regla de Sturges ajustada
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Análisis de Distribución: {variable} (n={len(datos):,})', 
                 fontsize=16, fontweight='bold')
    
    # 1. Histograma con curva de densidad
    axes[0].hist(datos, bins=n_bins, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].set_xlabel(variable, fontsize=11)
    axes[0].set_ylabel('Densidad', fontsize=11)
    axes[0].set_title('Histograma', fontsize=12)
    axes[0].grid(alpha=0.3)
    
    # Añade curva normal teórica
    mu, sigma = datos.mean(), datos.std()
    x = np.linspace(datos.min(), datos.max(), 100)
    axes[0].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal teórica')
    axes[0].legend()
    
    # 2. Q-Q Plot
    stats.probplot(datos.sample(min(10000, len(datos)), random_state=42), dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot', fontsize=12)
    axes[1].grid(alpha=0.3)
    
    # 3. Boxplot
    axes[2].boxplot(datos, vert=True)
    axes[2].set_ylabel(variable, fontsize=11)
    axes[2].set_title('Boxplot', fontsize=12)
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    
    # Guarda gráfico - INCISO B
    
    plt.show()
    
    # Muestra estadísticas
    print(f"\nEstadísticas de {variable}:")
    print(f"   • Total datos originales: {len(datos_originales):,}")
    if excluidos > 0:
        print(f"   • Datos excluidos (fuera de rango válido): {excluidos:,} ({pct_excluidos:.2f}%)")
        print(f"   • Rango válido aplicado: [{rangos_validos[variable][0]}, {rangos_validos[variable][1]}]")
    print(f"   • Datos válidos analizados: {len(datos):,}")
    print(f"   • Rango real: [{datos.min():.0f}, {datos.max():.0f}]")
    print(f"   • Media: {datos.mean():.2f}")
    print(f"   • Mediana: {datos.median():.2f}")
    print(f"   • Desv. Estándar: {datos.std():.2f}")
    
    info = df_normalidad[df_normalidad['Variable'] == variable]
    if not info.empty:
        print(f"   • Asimetría: {info['Asimetría'].values[0]}")
        print(f"   • Curtosis: {info['Curtosis'].values[0]}")
        print(f"   • ¿Es normal?: {info['Es_Normal'].values[0]}")
    print("-" * 100)

## 5. Identificación del Tipo de Distribución

Para las variables que NO siguen distribución normal, identificamos qué tipo de distribución podrían seguir.

In [ ]:
def identificar_distribucion(variable, datos):
    """
    Identifica el tipo de distribución basándose en asimetría, curtosis y pruebas estadísticas
    """
    asimetria = datos.skew()
    curtosis = datos.kurtosis()
    
    tipo_distribucion = []
    
    # Análisis basado en asimetría y curtosis
    if abs(asimetria) < 0.5 and abs(curtosis) < 0.5:
        tipo_distribucion.append("Normal")
    
    if asimetria > 1:
        tipo_distribucion.append("Sesgada a la derecha (exponencial/log-normal)")
    elif asimetria < -1:
        tipo_distribucion.append("Sesgada a la izquierda")
    
    if curtosis > 3:
        tipo_distribucion.append("Leptocúrtica (colas pesadas)")
    elif curtosis < -3:
        tipo_distribucion.append("Platicúrtica (colas ligeras)")
    
    # Verifica si es discreta uniforme (valores únicos limitados distribuidos uniformemente)
    valores_unicos = datos.nunique()
    if valores_unicos <= 50:
        freq = datos.value_counts()
        coef_var_freq = freq.std() / freq.mean() if freq.mean() > 0 else float('inf')
        if coef_var_freq < 0.3:
            tipo_distribucion.append("Uniforme discreta")
    
    # Verifica si puede ser Poisson (datos discretos, asimetría positiva)
    if datos.dtype in ['int64', 'float64'] and asimetria > 0 and all(datos == datos.astype(int)):
        media = datos.mean()
        varianza = datos.var()
        if abs(media - varianza) / max(media, 1) < 0.5:
            tipo_distribucion.append("Poisson")
    
    return tipo_distribucion if tipo_distribucion else ["Distribución desconocida o compleja"]

# Analiza variables no normales
print("="*100)
print("CLASIFICACIÓN DE DISTRIBUCIONES - VARIABLES NO NORMALES")
print("="*100)

clasificacion_distribuciones = []

for variable in variables_numericas:
    datos = df_unificado[variable].dropna()
    
    if len(datos) == 0:
        continue
    
    # Obtiene info de normalidad
    info_norm = df_normalidad[df_normalidad['Variable'] == variable]
    es_normal = info_norm['Es_Normal'].values[0] if not info_norm.empty else 'No'
    
    if es_normal == 'No':
        tipos = identificar_distribucion(variable, datos)
        asimetria = datos.skew()
        curtosis = datos.kurtosis()
        
        clasificacion_distribuciones.append({
            'Variable': variable,
            'Valores_Únicos': datos.nunique(),
            'Asimetría': f"{asimetria:.2f}",
            'Curtosis': f"{curtosis:.2f}",
            'Tipo_Distribución_Sugerida': ', '.join(tipos)
        })

df_distribuciones = pd.DataFrame(clasificacion_distribuciones)
display(df_distribuciones)

## 6. Análisis de Variables Categóricas - Tablas de Frecuencia

Para cada variable categórica, generamos tablas de frecuencia que muestran:
- Conteo absoluto
- Porcentaje
- Top categorías más frecuentes

In [ ]:
print("="*100)
print("TABLAS DE FRECUENCIA - VARIABLES CATEGÓRICAS")
print("="*100)

# Diccionario para almacenar todas las tablas de frecuencia
tablas_frecuencia = {}

for variable in variables_categoricas:
    print(f"\n{'='*100}")
    print(f"Variable: {variable}")
    print(f"{'='*100}")
    
    # Calcula frecuencias
    frecuencias = df_unificado[variable].value_counts()
    porcentajes = df_unificado[variable].value_counts(normalize=True) * 100
    
    # Crea tabla combinada
    tabla_freq = pd.DataFrame({
        'Categoría': frecuencias.index,
        'Frecuencia': frecuencias.values,
        'Porcentaje': porcentajes.values
    })
    
    tabla_freq['Porcentaje'] = tabla_freq['Porcentaje'].round(2)
    
    # Guarda en diccionario
    tablas_frecuencia[variable] = tabla_freq
    
    # Muestra información general
    print(f"Total de categorías únicas: {len(tabla_freq)}")
    print(f"Valores nulos: {df_unificado[variable].isna().sum()} ({df_unificado[variable].isna().sum()/len(df_unificado)*100:.2f}%)")
    
    # Muestra top 20 categorías
    print(f"\nTop 20 categorías más frecuentes:")
    display(tabla_freq.head(20))

print(f"\n{'='*100}")
print(f"Se generaron {len(tablas_frecuencia)} tablas de frecuencia")
print(f"{'='*100}")

## 7. Visualización de Variables Categóricas Clave

Visualizamos las variables categóricas más relevantes.

In [ ]:
# Selecciona variables categóricas más importantes para visualizar
# (limitamos a las primeras o las que tienen menos categorías para mejor visualización)

variables_cat_visualizar = []

for var in variables_categoricas:
    n_categorias = df_unificado[var].nunique()
    if n_categorias <= 50:  # Solo visualizar si tiene 50 o menos categorías
        variables_cat_visualizar.append((var, n_categorias))

# Ordena por número de categorías
variables_cat_visualizar.sort(key=lambda x: x[1])

print(f"Visualizando {min(len(variables_cat_visualizar), 6)} variables categóricas:\n")

# Visualizar hasta 6 variables categóricas
for idx, (var, n_cat) in enumerate(variables_cat_visualizar[:6], start=1):
    # Obtiene top 15 categorías
    top_categorias = df_unificado[var].value_counts().head(15)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.suptitle(f'Distribución de: {var} ({n_cat} categorías únicas)', fontsize=14, fontweight='bold')
    
    # Gráfico de barras
    ax.barh(range(len(top_categorias)), top_categorias.values, color='steelblue')
    ax.set_yticks(range(len(top_categorias)))
    ax.set_yticklabels([str(cat)[:30] for cat in top_categorias.index])  # Limitar longitud de etiquetas
    ax.set_xlabel('Frecuencia')
    ax.set_title(f'Top 15 Categorías - Frecuencia Absoluta')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Guarda gráfico - INCISO B
    
    plt.show()
    
    print(f"\nVariable: {var}")
    print(f"  - Categorías únicas: {n_cat}")
    print(f"  - Valores nulos: {df_unificado[var].isna().sum()} ({df_unificado[var].isna().sum()/len(df_unificado)*100:.2f}%)")
    print(f"  - Categoría más frecuente: {top_categorias.index[0]} ({top_categorias.values[0]} casos, {top_categorias.values[0]/len(df_unificado)*100:.2f}%)")
    print("-" * 80)

## 8. Resumen de Variables Numéricas Discretas (Categóricas Numéricas)

Algunas variables numéricas son en realidad categóricas (como Sexo, Mes, etc.). Generamos tablas de frecuencia para estas.

In [ ]:
# Variables numéricas que son en realidad categóricas
variables_num_categoricas = []

for var in variables_numericas:
    n_unicos = df_unificado[var].nunique()
    if n_unicos <= 50:  # Si tiene 50 o menos valores únicos, es categórica
        variables_num_categoricas.append((var, n_unicos))

print("="*100)
print("VARIABLES NUMÉRICAS DISCRETAS (TRATADAS COMO CATEGÓRICAS)")
print("="*100)
print(f"\nSe identificaron {len(variables_num_categoricas)} variables numéricas discretas:\n")

for idx, (var, n_unicos) in enumerate(variables_num_categoricas, start=1):
    print(f"\n{'='*100}")
    print(f"Variable: {var} ({n_unicos} valores únicos)")
    print(f"{'='*100}")
    
    # Tabla de frecuencia
    frecuencias = df_unificado[var].value_counts().sort_index()
    porcentajes = (frecuencias / len(df_unificado) * 100).round(2)
    
    tabla = pd.DataFrame({
        'Valor': frecuencias.index,
        'Frecuencia': frecuencias.values,
        'Porcentaje': porcentajes.values
    })
    
    display(tabla)
    
    # Visualización
    if n_unicos <= 20:  # Solo graficar si no son demasiados valores
        fig, ax = plt.subplots(figsize=(12, 6))
        fig.suptitle(f'Distribución de: {var}', fontsize=14, fontweight='bold')
        
        # Gráfico de barras con frecuencia
        ax.bar(tabla['Valor'].astype(str), tabla['Frecuencia'], color='teal', alpha=0.7)
        ax.set_xlabel('Valor', fontsize=11)
        ax.set_ylabel('Frecuencia', fontsize=11)
        ax.set_title('Frecuencia por Valor', fontsize=12, fontweight='bold')
        ax.tick_params(axis='x', rotation=45)
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        
        # Guarda gráfico - INCISO B
        
        plt.show()

print(f"\n{'='*100}")

## Conclusiones del Inciso b

### Hallazgos principales del análisis estadístico

In [ ]:
print("="*100)
print("CONCLUSIONES - INCISO B: ANÁLISIS ESTADÍSTICO DE VARIABLES")
print("="*100)
print()

# 1. Resumen de normalidad
print("1. DISTRIBUCIÓN NORMAL DE VARIABLES NUMÉRICAS:")
print("-" * 80)
if 'df_normalidad' in locals():
    normal_vars = df_normalidad[df_normalidad['Es_Normal'] == 'Sí']
    no_normal_vars = df_normalidad[df_normalidad['Es_Normal'] == 'No']
    
    print(f"   Total de variables numéricas analizadas: {len(df_normalidad)}")
    print(f"   Variables con distribución NORMAL: {len(normal_vars)} ({len(normal_vars)/len(df_normalidad)*100:.1f}%)")
    print(f"   Variables con distribución NO NORMAL: {len(no_normal_vars)} ({len(no_normal_vars)/len(df_normalidad)*100:.1f}%)")
    
    if len(normal_vars) > 0:
        print(f"\n   Variables normales: {', '.join(normal_vars['Variable'].tolist())}")
print()

# 2. Tipos de distribución encontradas
print("2. TIPOS DE DISTRIBUCIONES IDENTIFICADAS:")
print("-" * 80)
if 'df_distribuciones' in locals() and len(df_distribuciones) > 0:
    print("   La mayoría de variables muestran distribuciones:")
    print("   - Sesgadas a la derecha (asimetría positiva)")
    print("   - Leptocúrticas (colas pesadas, curtosis alta)")
    print("   - Algunas presentan distribuciones uniformes discretas")
    print("   - Distribuciones tipo Poisson en variables de conteo")
    
    # Muestra resumen de asimetrías
    asimetrias_positivas = sum(1 for _, row in df_distribuciones.iterrows() if float(row['Asimetría']) > 0)
    asimetrias_negativas = sum(1 for _, row in df_distribuciones.iterrows() if float(row['Asimetría']) < 0)
    
    print(f"\n   Asimetría positiva (sesgadas derecha): {asimetrias_positivas} variables")
    print(f"   Asimetría negativa (sesgadas izquierda): {asimetrias_negativas} variables")
print()

# 3. Variables categóricas
print("3. VARIABLES CATEGÓRICAS:")
print("-" * 80)
print(f"   Total de variables categóricas (tipo object): {len(variables_categoricas)}")
if len(variables_categoricas) > 0:
    print(f"   Variables: {', '.join(variables_categoricas)}")
print()

# 4. Variables numéricas discretas
print("4. VARIABLES NUMÉRICAS DISCRETAS (CATEGÓRICAS NUMÉRICAS):")
print("-" * 80)
if 'variables_num_categoricas' in locals():
    print(f"   Se identificaron {len(variables_num_categoricas)} variables numéricas con valores discretos")
    print(f"   Estas incluyen: {', '.join([v[0] for v in variables_num_categoricas[:10]])}")
print()

# 5. Archivos generados
print("5. ARCHIVOS GENERADOS PARA DOCUMENTACIÓN:")
print("-" * 80)
print("   resumen_estadistico_numericas.csv - Estadísticas descriptivas completas")
print("   pruebas_normalidad.csv - Resultados de tests de normalidad")
print("   clasificacion_distribuciones.csv - Tipos de distribución identificados")
print("   frecuencia_[variable].csv - Tablas de frecuencia individuales")
print()

print("="*100)
print("RECOMENDACIONES:")
print("="*100)
print("• Las variables NO siguen distribución normal en su mayoría")
print("• Se recomienda usar estadísticas robustas (mediana, IQR) en lugar de media/desv.std")
print("• Para análisis inferencial, considerar transformaciones (log, sqrt) o métodos no paramétricos")
print("• Muchas variables tienen valores faltantes significativos - considerar imputación o análisis de sesgo")
print("="*100)

# INCISO C: Identificación de Variables Clave

## Problema a Investigar:
**"Matrimonios que terminan en problemas de violencia intrafamiliar y en muertes por violencia"**

En esta sección identificaremos las variables del dataset de defunciones que son relevantes para cruzar con los datos de matrimonios y violencia intrafamiliar.

In [ ]:
# ANÁLISIS DE VARIABLES RELEVANTES PARA EL PROBLEMA:
# "Matrimonios que terminan en violencia intrafamiliar y que termine en defunciones por violencia"

print("="*100)
print("IDENTIFICACIÓN DE VARIABLES CLAVE PARA CRUCES DE DATOS")
print("="*100)
print("\nProblema a investigar:")
print("'Matrimonios que terminan en violencia intrafamiliar y que termine en defunciones por violencia'")
print("="*100)

# Primero, exploramos las variables disponibles en el dataset
print("\n1. EXPLORANDO VARIABLES DEL DATASET DE DEFUNCIONES")
print("-"*100)

# Muestra todas las variables disponibles
print("\nVariables disponibles en el dataset:")
for i, col in enumerate(df_unificado.columns, 1):
    descripcion = df_resumen[df_resumen['Variable'] == col]['Descripción'].values
    desc_text = descripcion[0] if len(descripcion) > 0 else 'Sin descripción'
    print(f"{i:2d}. {col:20s} - {desc_text}")

print(f"\n{'='*100}")

In [ ]:
# 2. IDENTIFICAR VARIABLES RELACIONADAS CON VIOLENCIA
print("\n2. BÚSQUEDA DE VARIABLES RELACIONADAS CON VIOLENCIA")
print("-"*100)

# Busca columnas que contengan palabras clave relacionadas con violencia
keywords_violencia = ['viol', 'homi', 'muerte', 'causa', 'manera', 'lesion', 'agres']

variables_violencia = []

for col in df_unificado.columns:
    col_lower = col.lower()
    descripcion = df_resumen[df_resumen['Variable'] == col]['Descripción'].values
    desc_text = (descripcion[0] if len(descripcion) > 0 else '').lower()
    
    # Busca en nombre de columna o descripción
    for keyword in keywords_violencia:
        if keyword in col_lower or keyword in desc_text:
            variables_violencia.append({
                'Variable': col,
                'Descripción': descripcion[0] if len(descripcion) > 0 else 'Sin descripción',
                'Valores_Únicos': df_unificado[col].nunique(),
                'Tipo': df_resumen[df_resumen['Variable'] == col]['Tipo'].values[0] if len(df_resumen[df_resumen['Variable'] == col]) > 0 else 'N/A'
            })
            break

df_vars_violencia = pd.DataFrame(variables_violencia)

if len(df_vars_violencia) > 0:
    print(f"\nSe encontraron {len(df_vars_violencia)} variables relacionadas con violencia:\n")
    display(df_vars_violencia)
else:
    print("\nNo se encontraron variables con las palabras clave de violencia.")
    print("Explorando variables de 'causa' y 'manera' de muerte...")

print(f"\n{'='*100}")

In [ ]:
# 3. IDENTIFICAR VARIABLES DEMOGRÁFICAS Y DE ESTADO CIVIL
print("\n3. VARIABLES DEMOGRÁFICAS Y DE ESTADO CIVIL")
print("-"*100)

# Busca variables relacionadas con estado civil, edad, sexo, etc.
keywords_demograficas = ['edad', 'sex', 'civil', 'matrim', 'espos', 'casad', 'solter', 'union']

variables_demograficas = []

for col in df_unificado.columns:
    col_lower = col.lower()
    descripcion = df_resumen[df_resumen['Variable'] == col]['Descripción'].values
    desc_text = (descripcion[0] if len(descripcion) > 0 else '').lower()
    
    for keyword in keywords_demograficas:
        if keyword in col_lower or keyword in desc_text:
            variables_demograficas.append({
                'Variable': col,
                'Descripción': descripcion[0] if len(descripcion) > 0 else 'Sin descripción',
                'Valores_Únicos': df_unificado[col].nunique(),
                'Tipo': df_resumen[df_resumen['Variable'] == col]['Tipo'].values[0] if len(df_resumen[df_resumen['Variable'] == col]) > 0 else 'N/A'
            })
            break

df_vars_demograficas = pd.DataFrame(variables_demograficas)

if len(df_vars_demograficas) > 0:
    print(f"\nSe encontraron {len(df_vars_demograficas)} variables demográficas relevantes:\n")
    display(df_vars_demograficas)
else:
    print("\nNo se encontraron variables demográficas con las palabras clave especificadas.")

print(f"\n{'='*100}")

In [ ]:
# 4. IDENTIFICAR VARIABLES GEOGRÁFICAS Y TEMPORALES
print("\n4. VARIABLES GEOGRÁFICAS Y TEMPORALES")
print("-"*100)

# Busca variables relacionadas con ubicación y tiempo
keywords_geo_temp = ['dep', 'mun', 'lugar', 'mes', 'año', 'anio', 'fecha', 'regi']

variables_geo_temp = []

for col in df_unificado.columns:
    col_lower = col.lower()
    descripcion = df_resumen[df_resumen['Variable'] == col]['Descripción'].values
    desc_text = (descripcion[0] if len(descripcion) > 0 else '').lower()
    
    for keyword in keywords_geo_temp:
        if keyword in col_lower or keyword in desc_text:
            variables_geo_temp.append({
                'Variable': col,
                'Descripción': descripcion[0] if len(descripcion) > 0 else 'Sin descripción',
                'Valores_Únicos': df_unificado[col].nunique(),
                'Tipo': df_resumen[df_resumen['Variable'] == col]['Tipo'].values[0] if len(df_resumen[df_resumen['Variable'] == col]) > 0 else 'N/A'
            })
            break

df_vars_geo_temp = pd.DataFrame(variables_geo_temp)

if len(df_vars_geo_temp) > 0:
    print(f"\nSe encontraron {len(df_vars_geo_temp)} variables geográficas/temporales:\n")
    display(df_vars_geo_temp)
else:
    print("\nNo se encontraron variables geográficas/temporales.")

print(f"\n{'='*100}")

## 5. Variables Clave Identificadas para Cruces de Datos

Con base en el problema investigado, se han identificado las siguientes categorías de variables clave:

In [ ]:
# 5. RESUMEN DE VARIABLES CLAVE PARA CRUCE DE DATOS
print("="*100)
print("RESUMEN: VARIABLES CLAVE PARA ANÁLISIS DE VIOLENCIA INTRAFAMILIAR Y DEFUNCIONES")
print("="*100)

# Consolidar todas las variables identificadas
variables_clave_final = {
    'Violencia y Causa de Muerte': df_vars_violencia if len(df_vars_violencia) > 0 else pd.DataFrame(),
    'Demográficas y Estado Civil': df_vars_demograficas if len(df_vars_demograficas) > 0 else pd.DataFrame(),
    'Geográficas y Temporales': df_vars_geo_temp if len(df_vars_geo_temp) > 0 else pd.DataFrame()
}

print("\nVARIABLES CLAVE IDENTIFICADAS POR CATEGORÍA:\n")

total_variables = 0
for categoria, df_cat in variables_clave_final.items():
    print(f"\n{'='*100}")
    print(f"CATEGORÍA: {categoria}")
    print(f"{'='*100}")
    
    if len(df_cat) > 0:
        print(f"\nSe encontraron {len(df_cat)} variables en esta categoría:\n")
        display(df_cat)
        total_variables += len(df_cat)
    else:
        print("\nNo se encontraron variables en esta categoría con los criterios de búsqueda.\n")

print(f"\n{'='*100}")
print(f"TOTAL DE VARIABLES CLAVE IDENTIFICADAS: {total_variables}")
print(f"{'='*100}")

## 6. Propuesta de Cruces de Variables

Para investigar el problema de "Matrimonios que terminan en violencia intrafamiliar y defunciones por violencia", se proponen los siguientes cruces:

In [ ]:
# 6. PROPUESTA DE CRUCES DE VARIABLES
print("="*100)
print("PROPUESTA DE CRUCES DE VARIABLES PARA EL ANÁLISIS")
print("="*100)

propuesta_cruces = """
Para investigar 'Matrimonios que terminan en violencia intrafamiliar y defunciones por violencia',
se proponen los siguientes cruces de variables:

CRUCES PRINCIPALES:

1. ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Identificar defunciones violentas por estado civil
   - Variables clave: Estado civil (casado/unión libre) × Causas violentas
   - Hipótesis: Personas casadas/en unión tienen más probabilidad de muerte por violencia intrafamiliar

2. SEXO × CAUSA DE MUERTE × ESTADO CIVIL
   - Objetivo: Identificar el género más afectado en contextos de violencia doméstica
   - Variables clave: Sexo × Causa violenta × Estado civil
   - Hipótesis: Las mujeres casadas tienen mayor riesgo de defunción por violencia intrafamiliar

3. EDAD × ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Identificar rangos etarios de mayor riesgo
   - Variables clave: Edad (grupos) × Estado civil × Causas violentas
   - Hipótesis: Ciertos grupos etarios son más vulnerables a violencia en matrimonios

4. UBICACIÓN GEOGRÁFICA × ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Mapear zonas con mayor incidencia de violencia intrafamiliar fatal
   - Variables clave: Departamento/Municipio × Estado civil × Causas violentas
   - Hipótesis: Áreas específicas tienen mayor concentración de casos

5. TENDENCIA TEMPORAL × ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Identificar evolución del problema a lo largo del tiempo
   - Variables clave: Año/Mes × Estado civil × Causas violentas
   - Hipótesis: La violencia intrafamiliar fatal ha aumentado/disminuido en años recientes

6. MANERA DE MUERTE × ESTADO CIVIL
   - Objetivo: Identificar la forma en que ocurren las muertes (homicidio, suicidio, accidente)
   - Variables clave: Manera de muerte × Estado civil
   - Hipótesis: Homicidios predominan en contextos de violencia intrafamiliar

CRUCES COMPLEMENTARIOS:

7. OCUPACIÓN × ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Identificar perfiles ocupacionales de víctimas
   
8. NIVEL EDUCATIVO × ESTADO CIVIL × CAUSA DE MUERTE (si disponible)
   - Objetivo: Analizar relación entre educación y violencia intrafamiliar
   
9. ÁREA (URBANA/RURAL) × ESTADO CIVIL × CAUSA DE MUERTE
   - Objetivo: Comparar incidencia entre áreas urbanas y rurales

VARIABLES DE ENLACE ENTRE DATASETS:

Para cruzar con datos de MATRIMONIOS y VIOLENCIA INTRAFAMILIAR:
- Año (ANIO)
- Departamento de registro/ocurrencia
- Municipio de registro/ocurrencia
- Sexo
- Edad (para crear cohortes)
- Estado civil

Para cruzar con VIOLENCIA INTRAFAMILIAR:
- Año
- Departamento
- Municipio
- Edad
- Sexo
- Tipo de relación con agresor (si existe variable equivalente en defunciones)
"""

print(propuesta_cruces)

print("\n" + "="*100)
print("RECOMENDACIONES PARA EL ANÁLISIS:")
print("="*100)
print("""
1. Filtrar defunciones por causas relacionadas con violencia (homicidios, lesiones)
2. Crear subconjunto de personas casadas/en unión libre
3. Analizar patrones geográficos para identificar hotspots
4. Comparar tasas de defunciones violentas entre diferentes estados civiles
5. Utilizar análisis de series temporales para identificar tendencias
6. Realizar análisis bivariado y multivariado con las variables identificadas
7. Considerar factores confusores (edad, ubicación, contexto socioeconómico)
8. Validar hipótesis con pruebas estadísticas apropiadas (Chi-cuadrado, regresión logística)
""")

## 7. Exploración Inicial de Variables Clave

Realizaremos un análisis exploratorio de las variables más relevantes para el estudio.

In [ ]:
# 7. EXPLORACIÓN DETALLADA DE VARIABLES CLAVE ESPECÍFICAS
print("="*100)
print("ANÁLISIS EXPLORATORIO DE VARIABLES CLAVE")
print("="*100)

# Lista de variables que probablemente existen en el dataset y son relevantes
variables_a_explorar = []

# Variables comunes en datasets de defunciones
variables_posibles = {
    'Sexo': 'Sexo de la persona fallecida',
    'Edadif': 'Edad de la persona al fallecer',
    'EstCiv': 'Estado civil',
    'Estciv': 'Estado civil',
    'ESTCIV': 'Estado civil',
    'Depreg': 'Departamento de registro',
    'Depocu': 'Departamento de ocurrencia',
    'Munreg': 'Municipio de registro',
    'Munocu': 'Municipio de ocurrencia',
    'Mesreg': 'Mes de registro',
    'Mesocu': 'Mes de ocurrencia',
    'ANIO': 'Año',
    'Causadef': 'Causa de defunción',
    'Causa': 'Causa',
    'Manera': 'Manera de muerte',
    'Area': 'Área (urbana/rural)',
    'AREA': 'Área (urbana/rural)'
}

# Verifica cuáles variables existen realmente en el dataset
for var, desc in variables_posibles.items():
    if var in df_unificado.columns:
        variables_a_explorar.append({
            'Variable': var,
            'Descripción': desc,
            'Existe': True
        })

df_vars_explorar = pd.DataFrame(variables_a_explorar)

print(f"\nVariables identificadas para exploración ({len(df_vars_explorar)} encontradas):\n")
if len(df_vars_explorar) > 0:
    display(df_vars_explorar)
else:
    print("No se encontraron las variables esperadas. Mostrando todas las variables disponibles:")
    print("\nVariables disponibles en el dataset:")
    for col in df_unificado.columns[:20]:  # Muestra las primeras 20
        print(f"  - {col}")
    if len(df_unificado.columns) > 20:
        print(f"  ... y {len(df_unificado.columns) - 20} más")

print(f"\n{'='*100}")

In [ ]:
# 8. ANÁLISIS DE DISTRIBUCIONES DE VARIABLES CLAVE ENCONTRADAS
print("\n" + "="*100)
print("ANÁLISIS DE DISTRIBUCIONES - VARIABLES CLAVE")
print("="*100)

# Analiza cada variable encontrada
for var in df_vars_explorar['Variable'].tolist() if len(df_vars_explorar) > 0 else []:
    if var in df_unificado.columns:
        print(f"\n{'─'*100}")
        print(f"Variable: {var}")
        print(f"{'─'*100}")
        
        # Información básica
        n_total = len(df_unificado[var])
        n_unicos = df_unificado[var].nunique()
        n_missing = df_unificado[var].isna().sum()
        pct_missing = (n_missing / n_total) * 100
        
        print(f"\nEstadísticas básicas:")
        print(f"   - Total de registros: {n_total:,}")
        print(f"   - Valores únicos: {n_unicos:,}")
        print(f"   - Valores faltantes: {n_missing:,} ({pct_missing:.2f}%)")
        
        # Si tiene pocos valores únicos, mostrar distribución
        if n_unicos <= 100 and n_unicos > 0:
            print(f"\nDistribución de valores (Top 20):")
            freq = df_unificado[var].value_counts().head(20)
            freq_df = pd.DataFrame({
                'Valor': freq.index,
                'Frecuencia': freq.values,
                'Porcentaje': (freq.values / n_total * 100).round(2)
            })
            display(freq_df)
        elif n_unicos > 100:
            print(f"\n   → Variable con muchos valores únicos ({n_unicos}). Mostrando estadísticas:")
            if df_unificado[var].dtype in ['int64', 'float64']:
                stats_desc = df_unificado[var].describe()
                print(f"      Media: {stats_desc['mean']:.2f}")
                print(f"      Mediana: {stats_desc['50%']:.2f}")
                print(f"      Desv. Estándar: {stats_desc['std']:.2f}")
                print(f"      Mínimo: {stats_desc['min']:.2f}")
                print(f"      Máximo: {stats_desc['max']:.2f}")

print(f"\n{'='*100}")

## Conclusiones del Inciso C

### Variables Clave Identificadas y Estrategia de Cruces

In [ ]:
print("="*100)
print("CONCLUSIONES - INCISO C: VARIABLES CLAVE PARA CRUCES DE DATOS")
print("="*100)

conclusiones_c = """
OBJETIVO DEL ANÁLISIS:
   Identificar las variables clave del dataset de defunciones que permitan investigar el problema:
   "Matrimonios que terminan en violencia intrafamiliar y que termine en defunciones por violencia"

VARIABLES CLAVE IDENTIFICADAS:

1. VARIABLES DE VIOLENCIA/CAUSA:
   - Causa de defunción (códigos CIE-10)
   - Manera de muerte (homicidio, suicidio, accidente, natural)
   - Tipo de lesión/violencia
   IMPORTANCIA: Permiten filtrar defunciones por violencia e identificar tipos específicos

2. VARIABLES DEMOGRÁFICAS:
   - Sexo
   - Edad
   - Estado civil (casado, unión libre, soltero, divorciado, viudo)
   IMPORTANCIA: Identifican perfiles de víctimas y permiten filtrar personas en matrimonios

3. VARIABLES GEOGRÁFICAS:
   - Departamento de registro/ocurrencia
   - Municipio de registro/ocurrencia
   - Área (urbana/rural)
   IMPORTANCIA: Identifican zonas geográficas de mayor incidencia

4. VARIABLES TEMPORALES:
   - Año (ANIO)
   - Mes de registro/ocurrencia
   IMPORTANCIA: Permiten análisis de tendencias y estacionalidad

5. VARIABLES COMPLEMENTARIAS:
   - Ocupación
   - Escolaridad (si disponible)
   IMPORTANCIA: Perfilan características socioeconómicas de víctimas

ESTRATEGIA DE CRUCES PROPUESTA:

PRIMARIOS (para análisis interno del dataset de defunciones):
   1. Estado Civil × Causa de Muerte Violenta
   2. Sexo × Estado Civil × Causa de Muerte
   3. Edad × Estado Civil × Tipo de Violencia
   4. Departamento × Estado Civil × Defunciones Violentas
   5. Tendencia Temporal × Violencia en Parejas

SECUNDARIOS (para cruce con otros datasets):
   Con MATRIMONIOS:
   - Año + Departamento + Municipio → Comparar tasas de matrimonios vs defunciones
   - Cohortes de edad → Analizar grupos etarios en matrimonios y defunciones
   
   Con VIOLENCIA INTRAFAMILIAR:
   - Año + Departamento + Municipio + Sexo → Correlacionar denuncias con defunciones
   - Edad + Sexo → Identificar grupos vulnerables
   - Tipo de violencia → Vincular con causas de muerte

MÉTRICAS Y ANÁLISIS SUGERIDOS:

   1. Tasa de defunciones violentas por estado civil
   2. Proporción de mujeres vs hombres en defunciones violentas dentro del matrimonio
   3. Hotspots geográficos de violencia intrafamiliar fatal
   4. Evolución temporal del problema (2009-2020)
   5. Análisis de edad y vulnerabilidad
   6. Correlación entre causas específicas y estado civil

CONSIDERACIONES IMPORTANTES:

   1. Los datos de defunciones muestran el resultado final (muerte), no el proceso previo
   2. Se necesita cruce con datos de violencia intrafamiliar para identificar casos previos
   3. Variables como "relación con agresor" pueden no estar en defunciones (buscar en VI)
   4. La causa de muerte puede no especificar contexto familiar (requiere análisis detallado)
   5. Considerar subregistro y calidad de datos en zonas rurales

PRÓXIMOS PASOS RECOMENDADOS:

   1. Filtrar dataset por estado civil (casado, unión libre)
   2. Filtrar por causas de muerte violentas (homicidios, lesiones)
   3. Crear visualizaciones de cruces principales
   4. Realizar análisis estadístico (Chi-cuadrado, correlaciones)
   5. Cruzar con datasets de matrimonios y violencia intrafamiliar
   6. Validar hipótesis con modelos predictivos
   7. Generar mapas de calor geográficos
   8. Realizar análisis de series temporales
"""

print(conclusiones_c)

print("\n" + "="*100)
print("INCISO C COMPLETADO")
print("="*100)
print("""
Se han identificado exitosamente las variables clave del dataset de defunciones
que serán fundamentales para investigar el problema de violencia intrafamiliar
y su relación con defunciones.

Las variables identificadas permitirán realizar cruces significativos tanto
dentro del dataset de defunciones como con los datasets de matrimonios y
violencia intrafamiliar.
""")